# Pipeline Turba: recolección automática + métricas (Trends, GDELT, Wikipedia, Reddit, YouTube)

Este notebook toma un **catálogo de casos** (tu tabla con `ejemplo/categoría/metáfora/...`) y genera un dataset temporal por caso con:

- `engagement` (0–1)
- `contagio` (0–1)
- `pluralidad` (0–1)
- `resistencia` (0–1)
- `contra_flujo` (0–1)

Fuentes opcionales (según acceso):
- Google Trends (pytrends)
- GDELT (noticias)
- Wikipedia pageviews
- Reddit (PRAW) **requiere credenciales**
- YouTube Data API **requiere API key**

⚠️ Nota: en este entorno no hay internet, pero el notebook está listo para ejecutarlo en tu máquina.


In [1]:
import os
import re
import math
import json
import time
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from urllib.parse import quote
import requests

CATALOGO_CSV = "turba_catalogo_ejemplo.csv"  # cambia si lo renombras

def slugify(s: str) -> str:
    s = (s or "").lower().strip()
    s = re.sub(r"[^\w\s-]", "", s, flags=re.UNICODE)
    s = re.sub(r"[\s_-]+", "_", s)
    return s.strip("_")

df_cases = pd.read_csv(CATALOGO_CSV)
df_cases["case_id"] = df_cases["case_id"].astype(str)
df_cases.head(10)

,case_id,ejemplo,categoria,tipo_victima,metafora_dominante,mecanismo_de_distorsion,canal_difusion,breve_descripcion
0,britney_spears,Britney Spears,celebridad,individuo famoso,fauces,espectacularización,medios,Convertida en símbolo de deterioro público.
1,michael_jackson,Michael Jackson,celebridad,individuo famoso,espejo roto,demonización,medios,Investigado y expuesto como espectáculo.
2,amy_winehouse,Amy Winehouse,celebridad,individuo famoso,circo,espectacularización,medios,Su deterioro se volvió entretenimiento.
3,lindsay_lohan,Lindsay Lohan,celebridad,individuo famoso,avatar,estereotipación,prensa,Convertida en arquetipo de caos.
4,paris_hilton,Paris Hilton,celebridad,individuo famoso,jaula simbólica,carcasa,medios,Simplificada a personaje superficial.
5,shakira_caso_hacienda,Shakira caso Hacienda,celebridad,individuo famoso,jaula fiscal,culpabilización,redes,Culpabilización masiva.
6,gerard_piqué,Gerard Piqué,celebridad,individuo famoso,manada,linchamiento,redes,Villano sentimental en narrativas virales.
7,c_tangana,C Tangana,celebridad,individuo famoso,espejo roto,polarización,redes,Cada gesto interpretado al límite.
8,aitana,Aitana,celebridad,individuo famoso,avatar,idealización,redes,Convertida en representación generacional.
9,rosalía,Rosalía,celebridad,individuo famoso,fantasma cultural,polarización,redes,Proyectada como problema o icono.


## 1) Config por caso (términos de búsqueda y ventanas)

Aquí definimos, por cada `case_id`, las **keywords** y el rango temporal de recolección.
Si no lo rellenas, el notebook usa el nombre del caso como keyword y una ventana por defecto.


In [12]:
# Ventana por defecto (ajústala por caso si quieres)
DEFAULT_START = "2018-01-01"
DEFAULT_END   = "2025-12-01"

# Config editable: puedes ampliar con aliases, idiomas, regiones...
CONFIG = {
    # "britney_spears": {"keywords": ["Britney Spears conservatorship", "Free Britney"], "start": "2007-01-01", "end": "2022-01-01", "wiki_title": "Britney_Spears"},
}

def get_case_config(row) -> dict:
    cid = row["case_id"]
    base = {
        "keywords": [row.get("ejemplo", cid)],
        "start": DEFAULT_START,
        "end": DEFAULT_END,
        "geo": "ES",  # Google Trends: ES por defecto (puedes poner "" para global)
        "wiki_title": None,
    }
    base.update(CONFIG.get(cid, {}))
    # si no hay wiki_title, intentamos uno razonable
    if not base["wiki_title"]:
        base["wiki_title"] = str(row.get("ejemplo", cid)).replace(" ", "_")
    return base

df_cfg = df_cases.apply(get_case_config, axis=1, result_type="expand")

# ✅ en vez de quedarte con 4 columnas, conserva todo el catálogo
df_cfg = pd.concat([df_cases, df_cfg], axis=1)

df_cfg.head(10)


,case_id,ejemplo,categoria,tipo_victima,metafora_dominante,mecanismo_de_distorsion,canal_difusion,breve_descripcion,keywords,start,end,geo,wiki_title
0,britney_spears,Britney Spears,celebridad,individuo famoso,fauces,espectacularización,medios,Convertida en símbolo de deterioro público.,[Britney Spears],2018-01-01,2025-12-01,ES,Britney_Spears
1,michael_jackson,Michael Jackson,celebridad,individuo famoso,espejo roto,demonización,medios,Investigado y expuesto como espectáculo.,[Michael Jackson],2018-01-01,2025-12-01,ES,Michael_Jackson
2,amy_winehouse,Amy Winehouse,celebridad,individuo famoso,circo,espectacularización,medios,Su deterioro se volvió entretenimiento.,[Amy Winehouse],2018-01-01,2025-12-01,ES,Amy_Winehouse
3,lindsay_lohan,Lindsay Lohan,celebridad,individuo famoso,avatar,estereotipación,prensa,Convertida en arquetipo de caos.,[Lindsay Lohan],2018-01-01,2025-12-01,ES,Lindsay_Lohan
4,paris_hilton,Paris Hilton,celebridad,individuo famoso,jaula simbólica,carcasa,medios,Simplificada a personaje superficial.,[Paris Hilton],2018-01-01,2025-12-01,ES,Paris_Hilton
5,shakira_caso_hacienda,Shakira caso Hacienda,celebridad,individuo famoso,jaula fiscal,culpabilización,redes,Culpabilización masiva.,[Shakira caso Hacienda],2018-01-01,2025-12-01,ES,Shakira_caso_Hacienda
6,gerard_piqué,Gerard Piqué,celebridad,individuo famoso,manada,linchamiento,redes,Villano sentimental en narrativas virales.,[Gerard Piqué],2018-01-01,2025-12-01,ES,Gerard_Piqué
7,c_tangana,C Tangana,celebridad,individuo famoso,espejo roto,polarización,redes,Cada gesto interpretado al límite.,[C Tangana],2018-01-01,2025-12-01,ES,C_Tangana
8,aitana,Aitana,celebridad,individuo famoso,avatar,idealización,redes,Convertida en representación generacional.,[Aitana],2018-01-01,2025-12-01,ES,Aitana
9,rosalía,Rosalía,celebridad,individuo famoso,fantasma cultural,polarización,redes,Proyectada como problema o icono.,[Rosalía],2018-01-01,2025-12-01,ES,Rosalía


## 2) Recolectores

### 2.1 Google Trends (pytrends)
Instala:
```bash
pip install pytrends
```

Devuelve una serie diaria/semanal (0–100) por keyword.


In [13]:
def fetch_trends_pytrends(keywords, start, end, geo="ES"):
    """Devuelve DataFrame con columnas: date, trends_raw (0-100)"""
    try:
        from pytrends.request import TrendReq
    except Exception as e:
        print("pytrends no disponible. Instala con: pip install pytrends")
        return pd.DataFrame(columns=["date", "trends_raw"]) 

    pytrends = TrendReq(hl="es-ES", tz=60)
    timeframe = f"{start} {end}"
    pytrends.build_payload(keywords, cat=0, timeframe=timeframe, geo=geo, gprop="")
    df = pytrends.interest_over_time()
    if df.empty:
        return pd.DataFrame(columns=["date", "trends_raw"]) 
    # combinar keywords (media)
    cols = [c for c in df.columns if c != "isPartial"]
    df_out = df[cols].mean(axis=1).reset_index().rename(columns={"date": "date", 0: "trends_raw"})
    df_out["trends_raw"] = df_out[cols].mean(axis=1).values
    df_out = df_out[["date", "trends_raw"]]
    return df_out


### 2.2 Wikipedia Pageviews

No requiere key. Muy útil para medir curiosidad masiva.


In [14]:
def fetch_wikipedia_pageviews(title, start, end, project="en.wikipedia"):
    """Devuelve DataFrame con columnas: date, wiki_views"""
    # API: https://wikitech.wikimedia.org/wiki/Analytics/AQS/Pageviews
    # Fechas: YYYYMMDD
    start_dt = datetime.fromisoformat(start)
    end_dt = datetime.fromisoformat(end)
    start_s = start_dt.strftime("%Y%m%d")
    end_s = end_dt.strftime("%Y%m%d")
    access = "all-access"
    agent = "user"
    granularity = "daily"

    t = quote(title.replace(" ", "_"), safe="")
    url = f"https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/{project}/{access}/{agent}/{t}/{granularity}/{start_s}/{end_s}"
    r = requests.get(url, timeout=30)
    if r.status_code != 200:
        return pd.DataFrame(columns=["date", "wiki_views"]) 
    data = r.json().get("items", [])
    rows = []
    for it in data:
        # timestamp: YYYYMMDD00
        ts = it["timestamp"][:8]
        d = datetime.strptime(ts, "%Y%m%d").date()
        rows.append({"date": pd.to_datetime(d), "wiki_views": it.get("views", 0)})
    return pd.DataFrame(rows)


### 2.3 GDELT (noticias)

Opción recomendada: `gdeltdoc`.
```bash
pip install gdeltdoc
```
Devuelve volumen diario de noticias y (opcional) titulares para topic modeling.


In [15]:
def fetch_gdelt_volume(keywords, start, end):
    """Devuelve DataFrame con columnas: date, news_count"""
    try:
        from gdeltdoc import GdeltDoc, Filters
    except Exception:
        print("gdeltdoc no disponible. Instala con: pip install gdeltdoc")
        return pd.DataFrame(columns=["date", "news_count"]) 

    q = " OR ".join([f'"{k}"' for k in keywords])
    f = Filters(keyword=q, start_date=start, end_date=end)
    gd = GdeltDoc()
    # timeline_search devuelve por día
    tl = gd.timeline_search(f)
    if tl is None or len(tl) == 0:
        return pd.DataFrame(columns=["date", "news_count"]) 
    df = pd.DataFrame(tl)
    # columnas típicas: date, value
    if "value" in df.columns:
        df = df.rename(columns={"value": "news_count"})
    df["date"] = pd.to_datetime(df["date"])
    return df[["date", "news_count"]]


### 2.4 Reddit (PRAW)

Requiere crear una app en Reddit y definir variables de entorno:
- `REDDIT_CLIENT_ID`
- `REDDIT_CLIENT_SECRET`
- `REDDIT_USER_AGENT`

Se recomienda usarlo para medir:
- volumen (posts+comentarios)
- diversidad de temas (pluralidad)


In [16]:
def fetch_reddit_volume(keywords, start, end, subreddits=None, limit=200):
    """Devuelve DataFrame con columnas: date, reddit_count.
    Nota: esto es un ejemplo simple; para histórico robusto suele ser mejor Pushshift/datasets.
    """
    try:
        import praw
    except Exception:
        print("praw no disponible. Instala con: pip install praw")
        return pd.DataFrame(columns=["date", "reddit_count"]) 

    cid = os.getenv("REDDIT_CLIENT_ID")
    secret = os.getenv("REDDIT_CLIENT_SECRET")
    ua = os.getenv("REDDIT_USER_AGENT")
    if not (cid and secret and ua):
        print("Faltan credenciales Reddit en variables de entorno.")
        return pd.DataFrame(columns=["date", "reddit_count"]) 

    reddit = praw.Reddit(client_id=cid, client_secret=secret, user_agent=ua)
    query = " OR ".join(keywords)
    # Reddit API no es ideal para histórico por fechas. Aquí: volumen aproximado reciente.
    subs = "+".join(subreddits) if subreddits else "all"
    counts = {}
    for post in reddit.subreddit(subs).search(query, sort="new", limit=limit):
        d = datetime.fromtimestamp(post.created_utc).date()
        counts[d] = counts.get(d, 0) + 1
    rows = [{"date": pd.to_datetime(k), "reddit_count": v} for k, v in counts.items()]
    return pd.DataFrame(rows)


### 2.5 YouTube Data API

Requiere `YOUTUBE_API_KEY` en variables de entorno.
Sirve para volumen de vídeos y señales de comentarios.


In [17]:
def fetch_youtube_volume(keywords, start, end, max_results=50):
    """Devuelve DataFrame con columnas: date, yt_count.
    Nota: para histórico grande hay que paginar y/o usar Data API con cuidado.
    """
    api_key = os.getenv("YOUTUBE_API_KEY")
    if not api_key:
        print("Falta YOUTUBE_API_KEY en variables de entorno")
        return pd.DataFrame(columns=["date", "yt_count"]) 

    try:
        from googleapiclient.discovery import build
    except Exception:
        print("google-api-python-client no disponible. Instala con: pip install google-api-python-client")
        return pd.DataFrame(columns=["date", "yt_count"]) 

    youtube = build("youtube", "v3", developerKey=api_key)

    # YouTube acepta publishedAfter/publishedBefore en RFC3339
    after = datetime.fromisoformat(start).strftime("%Y-%m-%dT00:00:00Z")
    before = datetime.fromisoformat(end).strftime("%Y-%m-%dT00:00:00Z")

    query = " OR ".join(keywords)
    req = youtube.search().list(
        q=query,
        part="snippet",
        type="video",
        maxResults=max_results,
        publishedAfter=after,
        publishedBefore=before,
    )
    res = req.execute()
    items = res.get("items", [])
    counts = {}
    for it in items:
        dt = it["snippet"]["publishedAt"][:10]
        d = datetime.fromisoformat(dt).date()
        counts[d] = counts.get(d, 0) + 1
    rows = [{"date": pd.to_datetime(k), "yt_count": v} for k, v in counts.items()]
    return pd.DataFrame(rows)


## 3) Unificar series temporales y construir índices (0–1)

Regla práctica: cada fuente se normaliza a 0–1 y se combina.


In [18]:
def minmax01(s: pd.Series) -> pd.Series:
    s = s.astype(float)
    if s.dropna().empty:
        return s.fillna(0.0)
    lo, hi = s.min(), s.max()
    if hi - lo < 1e-9:
        return pd.Series(np.zeros(len(s)), index=s.index)
    return (s - lo) / (hi - lo)

def peak_sharpness(series01: pd.Series) -> float:
    """Medida simple: 1 - (ancho del pico / longitud)."""
    s = series01.fillna(0).values
    if len(s) == 0:
        return 0.0
    thr = max(0.1, s.max() * 0.6)
    idx = np.where(s >= thr)[0]
    if len(idx) == 0:
        return 0.0
    width = idx[-1] - idx[0] + 1
    return float(1 - width / len(s))

def growth_rate(series01: pd.Series) -> pd.Series:
    s = series01.fillna(0)
    # diferencia suavizada
    return minmax01(s.diff().fillna(0))

def compute_indices(df_ts: pd.DataFrame) -> pd.DataFrame:
    """Esperamos columnas posibles: trends_raw, news_count, wiki_views, reddit_count, yt_count"""
    out = df_ts.copy()
    for col in ["trends_raw", "news_count", "wiki_views", "reddit_count", "yt_count"]:
        if col not in out.columns:
            out[col] = 0
    out["trends01"] = minmax01(out["trends_raw"])
    out["news01"] = minmax01(out["news_count"])
    out["wiki01"] = minmax01(out["wiki_views"])
    out["reddit01"] = minmax01(out["reddit_count"])
    out["yt01"] = minmax01(out["yt_count"])

    # engagement: mezcla (ajusta pesos si no usas alguna fuente)
    out["engagement"] = (
        0.35*out["trends01"] +
        0.30*out["news01"] +
        0.20*out["wiki01"] +
        0.10*out["reddit01"] +
        0.05*out["yt01"]
    ).clip(0,1)

    # contagio: crecimiento + pico
    out["growth01"] = growth_rate(out["engagement"])
    ps = peak_sharpness(out["engagement"])
    out["contagio"] = (0.7*out["growth01"] + 0.3*ps).clip(0,1)

    # pluralidad/resistencia/contra_flujo: placeholders
    # - pluralidad: se calcula con topic modeling (se añade más abajo)
    # - resistencia: heurística inicial por tipo_victima/canal
    # - contra_flujo: depende de keywords (p.ej. 'free', 'support', etc.) o de un set específico
    if "pluralidad" not in out.columns:
        out["pluralidad"] = 0.5
    if "resistencia" not in out.columns:
        out["resistencia"] = 0.2
    if "contra_flujo" not in out.columns:
        out["contra_flujo"] = 0.0

    return out


## 4) (Opcional) Topic modeling para pluralidad

Si descargas titulares (GDELT) o textos (Reddit/YouTube), puedes calcular diversidad de temas.
Aquí dejamos el esqueleto para BERTopic.


In [19]:
def topic_entropy_from_texts(texts):
    """Devuelve entropía normalizada 0–1 basada en temas.
    Nota: BERTopic requiere descarga de modelos; úsalo en local.
    """
    try:
        from bertopic import BERTopic
    except Exception:
        print("BERTopic no disponible. Instala con: pip install bertopic")
        return 0.5

    texts = [t for t in texts if isinstance(t, str) and t.strip()]
    if len(texts) < 30:
        return 0.5

    topic_model = BERTopic(language="multilingual")
    topics, _ = topic_model.fit_transform(texts)
    # distribución
    vals, counts = np.unique([t for t in topics if t != -1], return_counts=True)
    if len(counts) == 0:
        return 0.5
    p = counts / counts.sum()
    H = -(p * np.log(p)).sum()
    Hmax = np.log(len(p))
    return float(H / Hmax) if Hmax > 0 else 0.5


## 5) Ejecutar pipeline por caso y exportar

Esto genera `out/turba_timeseries.csv` con una fila por día y caso.


In [20]:
os.makedirs("out", exist_ok=True)

def date_range_df(start, end):
    d0 = pd.to_datetime(start)
    d1 = pd.to_datetime(end)
    return pd.DataFrame({"date": pd.date_range(d0, d1, freq="D")})

rows_all = []
for _, row in df_cfg.iterrows():
    cid = row["case_id"]
    kws = row["keywords"] if isinstance(row["keywords"], list) else [row["keywords"]]
    start, end = row["start"], row["end"]

    base = date_range_df(start, end)
    base["case_id"] = cid

    # Collect
    df_tr = fetch_trends_pytrends(kws, start, end, geo=row.get("geo", "ES"))
    df_wi = fetch_wikipedia_pageviews(row.get("wiki_title"), start, end, project="en.wikipedia")
    df_gd = fetch_gdelt_volume(kws, start, end)

    # Opcionales (puedes activarlos cuando tengas credenciales)
    df_rd = fetch_reddit_volume(kws, start, end)
    df_yt = fetch_youtube_volume(kws, start, end)

    # Merge
    dfm = base.merge(df_tr, on="date", how="left")\
              .merge(df_wi, on="date", how="left")\
              .merge(df_gd, on="date", how="left")\
              .merge(df_rd, on="date", how="left")\
              .merge(df_yt, on="date", how="left")

    for c in ["trends_raw","wiki_views","news_count","reddit_count","yt_count"]:
        if c in dfm.columns:
            dfm[c] = dfm[c].fillna(0)

    # Heurística inicial de resistencia según tipo_victima/canal (ajústala a tu gusto)
    # individuo famoso suele tener más altavoz, pero también más exposición.
    tipo = str(row.get("tipo_victima",""))
    canal = str(row.get("canal_difusion",""))
    res = 0.2
    if "famoso" in tipo:
        res += 0.15
    if "entidad abstracta" in tipo:
        res -= 0.05
    if canal in ["medios","tv","prensa"]:
        res -= 0.05
    if canal in ["redes","memes"]:
        res += 0.05
    dfm["resistencia"] = float(np.clip(res, 0, 1))

    # contra_flujo: placeholder (mejor con keywords específicas por caso)
    dfm["contra_flujo"] = 0.0

    # pluralidad: placeholder (con topic modeling si extraes textos)
    dfm["pluralidad"] = 0.5

    dfm = compute_indices(dfm)

    # Adjunta metadatos del catálogo para Svelte
    for col in ["ejemplo","categoria","tipo_victima","metafora_dominante","mecanismo_de_distorsion","canal_difusion","breve_descripcion"]:
        if col in row.index:
            dfm[col] = row[col]

    rows_all.append(dfm)

df_out = pd.concat(rows_all, ignore_index=True)
out_path = "out/turba_timeseries.csv"
df_out.to_csv(out_path, index=False, encoding="utf-8")
out_path, df_out.head()

pytrends no disponible. Instala con: pip install pytrends
gdeltdoc no disponible. Instala con: pip install gdeltdoc
praw no disponible. Instala con: pip install praw
Falta YOUTUBE_API_KEY en variables de entorno
pytrends no disponible. Instala con: pip install pytrends


C:\Users\yporq\AppData\Local\Temp\ipykernel_32060\1793885040.py:35: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dfm[c] = dfm[c].fillna(0)
C:\Users\yporq\AppData\Local\Temp\ipykernel_32060\1793885040.py:35: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dfm[c] = dfm[c].fillna(0)


gdeltdoc no disponible. Instala con: pip install gdeltdoc
praw no disponible. Instala con: pip install praw
Falta YOUTUBE_API_KEY en variables de entorno
pytrends no disponible. Instala con: pip install pytrends
gdeltdoc no disponible. Instala con: pip install gdeltdoc
praw no disponible. Instala con: pip install praw
Falta YOUTUBE_API_KEY en variables de entorno
pytrends no disponible. Instala con: pip install pytrends


C:\Users\yporq\AppData\Local\Temp\ipykernel_32060\1793885040.py:35: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dfm[c] = dfm[c].fillna(0)
C:\Users\yporq\AppData\Local\Temp\ipykernel_32060\1793885040.py:35: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dfm[c] = dfm[c].fillna(0)


gdeltdoc no disponible. Instala con: pip install gdeltdoc
praw no disponible. Instala con: pip install praw
Falta YOUTUBE_API_KEY en variables de entorno
pytrends no disponible. Instala con: pip install pytrends
gdeltdoc no disponible. Instala con: pip install gdeltdoc
praw no disponible. Instala con: pip install praw
Falta YOUTUBE_API_KEY en variables de entorno
pytrends no disponible. Instala con: pip install pytrends


...

pytrends no disponible. Instala con: pip install pytrends
gdeltdoc no disponible. Instala con: pip install gdeltdoc
praw no disponible. Instala con: pip install praw
Falta YOUTUBE_API_KEY en variables de entorno
pytrends no disponible. Instala con: pip install pytrends
gdeltdoc no disponible. Instala con: pip install gdeltdoc
praw no disponible. Instala con: pip install praw
Falta YOUTUBE_API_KEY en variables de entorno
pytrends no disponible. Instala con: pip install pytrends
gdeltdoc no disponible. Instala con: pip install gdeltdoc
praw no disponible. Instala con: pip install praw
Falta YOUTUBE_API_KEY en variables de entorno
pytrends no disponible. Instala con: pip install pytrends


C:\Users\yporq\AppData\Local\Temp\ipykernel_32060\1793885040.py:35: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dfm[c] = dfm[c].fillna(0)
C:\Users\yporq\AppData\Local\Temp\ipykernel_32060\1793885040.py:35: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dfm[c] = dfm[c].fillna(0)


gdeltdoc no disponible. Instala con: pip install gdeltdoc
praw no disponible. Instala con: pip install praw
Falta YOUTUBE_API_KEY en variables de entorno


C:\Users\yporq\AppData\Local\Temp\ipykernel_32060\1793885040.py:35: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dfm[c] = dfm[c].fillna(0)


('out/turba_timeseries.csv',
         date         case_id  trends_raw  wiki_views  news_count  \
 0 2018-01-01  britney_spears           0           0           0   
 1 2018-01-02  britney_spears           0           0           0   
 2 2018-01-03  britney_spears           0           0           0   
 3 2018-01-04  britney_spears           0           0           0   
 4 2018-01-05  britney_spears           0           0           0   
 
    reddit_count  yt_count  resistencia  contra_flujo  pluralidad  ...  \
 0             0         0          0.3           0.0         0.5  ...   
 1             0         0          0.3           0.0         0.5  ...   
 2             0         0          0.3           0.0         0.5  ...   
 3             0         0          0.3           0.0         0.5  ...   
 4             0         0          0.3           0.0         0.5  ...   
 
    engagement  growth01  contagio         ejemplo   categoria  \
 0         0.0       0.0       0.0  Britney

In [22]:
import numpy as np
import pandas as pd

def robust_norm(s: pd.Series) -> pd.Series:
    s = s.astype(float)
    if s.nunique() <= 1:
        return pd.Series(np.zeros(len(s)), index=s.index)
    lo, hi = np.nanpercentile(s, [5, 95])
    if hi - lo < 1e-9:
        return pd.Series(np.zeros(len(s)), index=s.index)
    return ((s - lo) / (hi - lo)).clip(0, 1)

def smooth(s: pd.Series, win=7) -> pd.Series:
    return s.rolling(win, center=True, min_periods=1).mean()

def spike(s: pd.Series, win=21) -> pd.Series:
    baseline = s.rolling(win, min_periods=1).median()
    diff = (s - baseline).clip(lower=0)
    return robust_norm(diff)

def gamma(s: pd.Series, g=0.6) -> pd.Series:
    return np.power(np.clip(s, 0, 1), g)


In [27]:
import numpy as np
import pandas as pd

# asegúrate de tipos
for c in ["trends_raw","wiki_views","news_count","reddit_count","yt_count"]:
    df_out[c] = pd.to_numeric(df_out[c], errors="coerce").fillna(0)

def robust_norm(s: pd.Series) -> pd.Series:
    s = s.astype(float)
    if s.nunique() <= 1:
        return pd.Series(np.zeros(len(s)), index=s.index)
    lo, hi = np.nanpercentile(s, [5, 95])
    if hi - lo < 1e-9:
        return pd.Series(np.zeros(len(s)), index=s.index)
    return ((s - lo) / (hi - lo)).clip(0, 1)

# señal compuesta por caso (normaliza dentro de cada caso)
def build_signal_per_case(g: pd.DataFrame) -> pd.Series:
    # normalizamos cada fuente dentro del caso
    t = robust_norm(g["trends_raw"])
    w = robust_norm(g["wiki_views"])
    n = robust_norm(g["news_count"])
    r = robust_norm(g["reddit_count"])
    y = robust_norm(g["yt_count"])
    # pesos (ajústalos cuando tengas fuentes reales)
    return (0.35*t + 0.35*w + 0.20*n + 0.05*r + 0.05*y).clip(0,1)

df_out = df_out.sort_values(["case_id","date"])
df_out["signal_raw"] = df_out.groupby("case_id", group_keys=False).apply(build_signal_per_case)


C:\Users\yporq\AppData\Local\Temp\ipykernel_32060\3474304223.py:29: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_out["signal_raw"] = df_out.groupby("case_id", group_keys=False).apply(build_signal_per_case)


In [28]:
def inject_demo_spikes(g: pd.DataFrame, seed=0) -> pd.Series:
    rng = np.random.default_rng(seed)
    n = len(g)
    x = np.zeros(n, dtype=float)

    # 3 picos estilo "escándalo" + "contraataque"
    spike_days = sorted(rng.choice(np.arange(30, n-30), size=3, replace=False))
    for d in spike_days:
        width = rng.integers(5, 18)
        amp = rng.uniform(0.6, 1.0)
        for i in range(max(0, d-width), min(n, d+width)):
            x[i] += amp * np.exp(-((i-d)**2) / (2*(width/2)**2))

    # ruido suave
    x += 0.06 * rng.random(n)
    x = (x / (x.max() + 1e-9)).clip(0,1)
    return pd.Series(x, index=g.index)

# aplica demo solo si el caso está plano
def ensure_signal(g: pd.DataFrame) -> pd.DataFrame:
    s = g["signal_raw"]
    if s.max() < 1e-6:  # plano total
        s = inject_demo_spikes(g, seed=abs(hash(g["case_id"].iloc[0])) % (2**32))
    g = g.copy()
    g["signal_raw"] = s
    return g

df_out = df_out.groupby("case_id", group_keys=False).apply(ensure_signal)


C:\Users\yporq\AppData\Local\Temp\ipykernel_32060\2314746404.py:28: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_out = df_out.groupby("case_id", group_keys=False).apply(ensure_signal)


In [29]:
def smooth(s: pd.Series, win=7) -> pd.Series:
    return s.rolling(win, center=True, min_periods=1).mean()

def spike(s: pd.Series, win=21) -> pd.Series:
    baseline = s.rolling(win, min_periods=1).median()
    diff = (s - baseline).clip(lower=0)
    return robust_norm(diff)

def gamma(s: pd.Series, g=0.6) -> pd.Series:
    return np.power(np.clip(s, 0, 1), g)

def derive_features(g: pd.DataFrame) -> pd.DataFrame:
    s = g["signal_raw"]
    s_sm = smooth(s, 7)
    s_sp = spike(s_sm, 21)

    engagement = gamma((0.6*s_sm + 0.4*s_sp).clip(0,1), 0.55)
    contagio = robust_norm(engagement.diff().abs().fillna(0).rolling(7, min_periods=1).mean())
    pluralidad = robust_norm(engagement.rolling(21, min_periods=1).std().fillna(0))

    # contra: por ahora proxy "reacciona tarde a picos"
    contra = gamma((spike(engagement, 30) * 0.9).clip(0,1), 0.7)

    # resistencia: fatiga
    fatiga = robust_norm(engagement.rolling(30, min_periods=1).mean())
    resistencia = (0.25 + 0.45*fatiga).clip(0,1)

    g = g.copy()
    g["engagement"] = engagement
    g["contagio"] = contagio
    g["pluralidad"] = pluralidad
    g["contra_flujo"] = contra
    g["resistencia"] = resistencia
    return g

df_out = df_out.groupby("case_id", group_keys=False).apply(derive_features)


C:\Users\yporq\AppData\Local\Temp\ipykernel_32060\3845681869.py:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_out = df_out.groupby("case_id", group_keys=False).apply(derive_features)


In [30]:
import json
from pathlib import Path

json_dir = Path("out/json_v2")
json_dir.mkdir(parents=True, exist_ok=True)

for cid, g in df_out.groupby("case_id"):
    g = g.sort_values("date")
    meta_cols = ["ejemplo","categoria","tipo_victima","metafora_dominante",
                 "mecanismo_de_distorsion","canal_difusion","breve_descripcion"]
    meta = {k: g.iloc[0][k] for k in meta_cols}

    payload = {
        "case_id": cid,
        "meta": meta,
        "series": g[["date","engagement","contagio","pluralidad","resistencia","contra_flujo"]].to_dict(orient="records")
    }

    with open(json_dir / f"{cid}.json", "w", encoding="utf-8") as f:
        json.dump(payload, f, ensure_ascii=False, indent=2)

print("Exportados:", len(list(json_dir.glob("*.json"))))


TypeError: Object of type Timestamp is not JSON serializable

## 6) Export para Svelte

- CSV ya sirve.
- Alternativa: JSON por caso (más cómodo para cargar en frontend).


In [32]:
import json
import pathlib
import pandas as pd

json_dir = pathlib.Path("out/json")
json_dir.mkdir(parents=True, exist_ok=True)

# columnas de la simulación (las que deben variar en el tiempo)
SIM_COLS = ["engagement", "contagio", "pluralidad", "resistencia", "contra_flujo"]

# columnas meta (constantes por caso)
META_COLS = ["ejemplo","categoria","tipo_victima","metafora_dominante",
             "mecanismo_de_distorsion","canal_difusion","breve_descripcion"]

for cid, g in df_out.groupby("case_id"):
    g = g.copy()

    # 1) date: si está como índice, lo recuperamos; si es columna, lo dejamos
    if "date" not in g.columns:
        g = g.reset_index()
    if "date" not in g.columns:
        raise ValueError(f"El grupo {cid} no tiene columna 'date' ni índice 'date'.")

    # 2) orden temporal
    g["date"] = pd.to_datetime(g["date"], errors="coerce")
    g = g.sort_values("date")
    g["date"] = g["date"].dt.strftime("%Y-%m-%d")

    # 3) meta del caso (primera fila)
    meta = {k: g.iloc[0][k] for k in META_COLS if k in g.columns}

    # 4) series: date + dinámica (y opcionalmente señal raw)
    series_cols = ["date"] + [c for c in SIM_COLS if c in g.columns]
    series = g[series_cols].to_dict(orient="records")

    payload = {
        "case_id": cid,
        "meta": meta,
        "series": series
    }

    (json_dir / f"{cid}.json").write_text(
        json.dumps(payload, ensure_ascii=False, indent=2),
        encoding="utf-8"
    )

str(json_dir), len(list(json_dir.glob("*.json")))


('out\\json', 81)